# <center> IMDB MOVIE REVIEW <center>

<hr style="border:2px  solid black"> </hr>

### TABLE OF CONTENTS
    
1. [Importing the Libraries](#lib_import)
2. [Exploring the 'IMDB Movie Review' Dataset](#explore-ds)
3. [Data Preparation](#data_prep)
      
    * [Reducing dataset size to 6k](#size)
    * [Retrieving percentage of punctuation and text of the original data](#%)
    * [Remove punctuations](#punc)
    * [Tokenization](#token)
    * [Remove stopwords](#stop)
    * [Stemming](#stem)
    * [Lemmatizing](#lemma)


4. [Data Modelling](#dm)
    * [Train_test_split](#tts)
    * [Grid searchCV](#gscv)
    * [Model evaluation](#me)
5. [Conclusion](#conc)

<hr style="border:.1px  solid  gray"> </hr>

## <CENTER> IMPORTING THE LIBRARIES<a name="lib_import"></a>


<hr style="border:01px  solid gray"> </hr>

In [1]:
#importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
import string

#import algorithms for modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

#import gridsearchCV and evaluation libraries
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_recall_fscore_support as score


In [2]:
#read the csv file
dataFrame = pd.read_csv("IMDB_dataset.csv")

In [3]:
dataFrame.head()

,review,sentiment
0,I thought this was a wonderful way to spend ti...,positive
1,"Probably my all-time favorite movie, a story o...",positive
2,I sure would like to see a resurrection of a u...,positive
3,"This show was an amazing, fresh & innovative i...",negative
4,Encouraged by the positive comments about this...,negative


<hr style="border:01px  solid gray"> </hr>

## <center>EXPLORING THE IMDB MOVIE REVIEW DATASET<a name="explore-ds"></a>
    

<hr style="border:01px  solid gray"> </hr>

Here, we would be exploring the dataset in general, like the below: 

* Shape, column names and size of the dataset
* Unique values of target variable `sentiment`. 
* Descriptive statistics.
* Datatype and overview . 
* Count of values of each feature and the target variable.

In [4]:
#get the shape
dataFrame.shape

(25000, 2)

In [5]:
# Check for null values
dataFrame.isnull().sum()

review       0
sentiment    0
dtype: int64

In [6]:
#get unique value_Counts 
dataFrame['sentiment'].value_counts()

positive    12500
negative    12500
Name: sentiment, dtype: int64

In [7]:
#get descriptive statistics
dataFrame.describe().T

,count,unique,top,freq
review,25000,24898,Loved today's show!!! It was a variety and not...,3
sentiment,25000,2,positive,12500


In [8]:
#get column names
dataFrame.columns

Index(['review', 'sentiment'], dtype='object')

In [9]:
#get info
dataFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     25000 non-null  object
 1   sentiment  25000 non-null  object
dtypes: object(2)
memory usage: 390.8+ KB


***

**OBSERVATION**

* From the above cells, we infer that the IMDB movie dataset has 25000 rows and 2 columns in `cell 4`.
* In `cell 5` , null values are checked for. We can see dataframe returns False indicating no null values and NaN.
* `Cell 6`, shows the `value_counts` found in the target variable `Sentiment`. `Sentiment` is a binary classification having `positive` and `negative` and is balanced. Hence sampling isn't needed.
* In `cell 7`, `describe()` , shows the descriptive statistics of the data set. 
* In `cell 8`, the column names are retreived.
* The datatype is obtained from the `info()` keyword,in `cell9` which states they are non-numeric of object type.

***

<hr style="border:01px  solid gray"> </hr>

## <center>DATA PREPARATION <a name="data_prep"></a>

<hr style="border:01px  solid gray"> </hr>


In this section, we will be performing data preparation steps like the below: 
1. [Reducing dataset size to 6k](#size)
2. [Retrieving percentage of punctuation and text of original data](#%)
3. [Remove punctuations](#punc)
4. [Tokenization](#token)
5. [Remove stopwords](#stop)
6. [ Stemming](#stem)
7. [Lemmatizing](#lemma)


***

### 1. Reducing the dataset size to 6000 records:

<a name="size"></a>

In [10]:
# selecting only first 3000 'positive' 
pos1 = dataFrame[dataFrame['sentiment'] == 'positive'].iloc[:3000]


In [11]:
# selecting only first 3000 'negative' 
neg1 = dataFrame[dataFrame['sentiment'] == 'negative'].iloc[:3000]

In [12]:
# appending pos1 and neg1
df = pos1.append(neg1)

In [13]:
#get new appended value_counts
df['sentiment'].value_counts()

positive    3000
negative    3000
Name: sentiment, dtype: int64

In [14]:
df.head()

,review,sentiment
0,I thought this was a wonderful way to spend ti...,positive
1,"Probably my all-time favorite movie, a story o...",positive
2,I sure would like to see a resurrection of a u...,positive
8,This a fantastic movie of three prisoners who ...,positive
11,"What an absolutely stunning movie, if you have...",positive


In [15]:
#Displaying maximum col width
pd.set_option('display.max_colwidth', None)

In [16]:
df.head()

,review,sentiment
0,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her ""sexy"" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than ""Devil Wears Prada"" and more interesting than ""Superman"" a great comedy to go see with friends.",positive
1,"Probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a noble cause, but it's not preachy or boring. It just never gets old, despite my having seen it some 15 or more times in the last 25 years. Paul Lukas' performance brings tears to my eyes, and Bette Davis, in one of her very few truly sympathetic roles, is a delight. The kids are, as grandma says, more like ""dressed-up midgets"" than children, but that only makes them more fun to watch. And the mother's slow awakening to what's happening in the world and under her own roof is believable and startling. If I had a dozen thumbs, they'd all be ""up"" for this movie.",positive
2,"I sure would like to see a resurrection of a up dated Seahunt series with the tech they have today it would bring back the kid excitement in me.I grew up on black and white TV and Seahunt with Gunsmoke were my hero's every week.You have my vote for a comeback of a new sea hunt.We need a change of pace in TV and this would work for a world of under water adventure.Oh by the way thank you for an outlet like this to view many viewpoints about TV and the many movies.So any ole way I believe I've got what I wanna say.Would be nice to read some more plus points about sea hunt.If my rhymes would be 10 lines would you let me submit,or leave me out to be in doubt and have me to quit,If this is so then I must go so lets do it.",positive
8,This a fantastic movie of three prisoners who become famous. One of the actors is george clooney and I'm not a fan but this roll is not bad. Another good thing about the movie is the soundtrack (The man of constant sorrow). I recommand this movie to everybody. Greetings Bart,positive
11,"What an absolutely stunning movie, if you have 2.5 hrs to kill, watch it, you won't regret it, it's too much fun! Rajnikanth carries the movie on his shoulders and although there isn't anything more other than him, I still liked it. The music by A.R.Rehman takes time to grow on you but after you heard it a few times, you really start liking it.",positive


In [17]:
df1 = df.copy()

<hr style="height:1px">

**OBSERVATION**

* In `cell 10` and `cell 11` , we reduce the dataset size to 3000 in each,Positive and negative, for swifter executions and to avoid memory error.
* From `cell 13` , we infer that the IMDB movie review dataset has reduced rows to 6000.  
* In `cell 4`, the column names are retreived.
* In `cell 15`  , `'display.max_colwidth` , is set to view the whole text . 
* In `cell 17` , the a copy of the original df is made to perform funtions like `removal_of_punctuations` , `tokenization` , `stemming` , `lemmatizing` & `removal_of_stopwords` for our understanding.

<hr style="height:1px">

### 2. Retrieving percentage of punctuation and text of the original data:

<a name="%"></a>

In [18]:
#Retreiving percentage count punc and text
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

df1['body_len'] = df1['review'].apply(lambda x: len(x) - x.count(" "))
df1['punct%'] = df1['review'].apply(lambda x: count_punct(x))

In [19]:
df1.head()

,review,sentiment,body_len,punct%
0,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her ""sexy"" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than ""Devil Wears Prada"" and more interesting than ""Superman"" a great comedy to go see with friends.",positive,761,5.3
1,"Probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a noble cause, but it's not preachy or boring. It just never gets old, despite my having seen it some 15 or more times in the last 25 years. Paul Lukas' performance brings tears to my eyes, and Bette Davis, in one of her very few truly sympathetic roles, is a delight. The kids are, as grandma says, more like ""dressed-up midgets"" than children, but that only makes them more fun to watch. And the mother's slow awakening to what's happening in the world and under her own roof is believable and startling. If I had a dozen thumbs, they'd all be ""up"" for this movie.",positive,538,5.2
2,"I sure would like to see a resurrection of a up dated Seahunt series with the tech they have today it would bring back the kid excitement in me.I grew up on black and white TV and Seahunt with Gunsmoke were my hero's every week.You have my vote for a comeback of a new sea hunt.We need a change of pace in TV and this would work for a world of under water adventure.Oh by the way thank you for an outlet like this to view many viewpoints about TV and the many movies.So any ole way I believe I've got what I wanna say.Would be nice to read some more plus points about sea hunt.If my rhymes would be 10 lines would you let me submit,or leave me out to be in doubt and have me to quit,If this is so then I must go so lets do it.",positive,577,2.1
8,This a fantastic movie of three prisoners who become famous. One of the actors is george clooney and I'm not a fan but this roll is not bad. Another good thing about the movie is the soundtrack (The man of constant sorrow). I recommand this movie to everybody. Greetings Bart,positive,226,3.1
11,"What an absolutely stunning movie, if you have 2.5 hrs to kill, watch it, you won't regret it, it's too much fun! Rajnikanth carries the movie on his shoulders and although there isn't anything more other than him, I still liked it. The music by A.R.Rehman takes time to grow on you but after you heard it a few times, you really start liking it.",positive,282,5.3


<hr style="height:1px">

**OBSERVATION**

* Prior cleaning the dataset in `Review` column, , punctuation percentage and length of the text in feature `Review` is retrived excluding the whitespaces in `cell 18` .

<hr style="height:1px">

### 3. Remove punctuations:
<a name="punc"></a>

In [20]:
df2 = df.copy()

In [21]:
# punc removal function definition
def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct

df2['body_text_clean_punc'] = df2['review'].apply(lambda x: remove_punct(x))

df2.head()

,review,sentiment,body_text_clean_punc
0,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her ""sexy"" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than ""Devil Wears Prada"" and more interesting than ""Superman"" a great comedy to go see with friends.",positive,I thought this was a wonderful way to spend time on a too hot summer weekend sitting in the air conditioned theater and watching a lighthearted comedy The plot is simplistic but the dialogue is witty and the characters are likable even the well bread suspected serial killer While some may be disappointed when they realize this is not Match Point 2 Risk Addiction I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to lovebr br This was the most Id laughed at one of Woodys comedies in years dare I say a decade While Ive never been impressed with Scarlet Johanson in this she managed to tone down her sexy image and jumped right into a average but spirited young womanbr br This may not be the crown jewel of his career but it was wittier than Devil Wears Prada and more interesting than Superman a great comedy to go see with friends
1,"Probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a noble cause, but it's not preachy or boring. It just never gets old, despite my having seen it some 15 or more times in the last 25 years. Paul Lukas' performance brings tears to my eyes, and Bette Davis, in one of her very few truly sympathetic roles, is a delight. The kids are, as grandma says, more like ""dressed-up midgets"" than children, but that only makes them more fun to watch. And the mother's slow awakening to what's happening in the world and under her own roof is believable and startling. If I had a dozen thumbs, they'd all be ""up"" for this movie.",positive,Probably my alltime favorite movie a story of selflessness sacrifice and dedication to a noble cause but its not preachy or boring It just never gets old despite my having seen it some 15 or more times in the last 25 years Paul Lukas performance brings tears to my eyes and Bette Davis in one of her very few truly sympathetic roles is a delight The kids are as grandma says more like dressedup midgets than children but that only makes them more fun to watch And the mothers slow awakening to whats happening in the world and under her own roof is believable and startling If I had a dozen thumbs theyd all be up for this movie
2,"I sure would like to see a resurrection of a up dated Seahunt series with the tech they have today it would bring back the kid excitement in me.I grew up on black and white TV and Seahunt with Gunsmoke were my hero's every week.You have my vote for a comeback of a new sea hunt.We need a change of pace in TV and this would work for a world of under water adventure.Oh by the way thank you for an outlet like this to view many viewpoints about TV and the many movies.So any ole way I believe I've got what I wanna say.Would be nice to read some more plus points about sea hunt.If my rhymes would be 10 lines would you let me submit,or leave me out to be in doubt and have me to quit,If this is so then I must go so lets do it.",positive,I sure would like to see a resurrection of a up dated Seahunt series with the tech th

<hr style="height:1px">

**OBSERVATION**

* Punctuations in `Review` column is removed using `remove_punct` function . 
* The new values without ounctuations are stored in `body_text_clean_punc`. 

<hr style="height:1px">

### 4. Tokenization:

<a name="token"></a>

In [22]:
#applying tokenization 
import re

def tokenize(text):
    tokens = re.split('\W+', text)
    return tokens

df2['body_text_tokenized'] = df2['body_text_clean_punc'].apply(lambda x: tokenize(x.lower()))

df2.head()

,review,sentiment,body_text_clean_punc,body_text_tokenized
0,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her ""sexy"" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than ""Devil Wears Prada"" and more interesting than ""Superman"" a great comedy to go see with friends.",positive,I thought this was a wonderful way to spend time on a too hot summer weekend sitting in the air conditioned theater and watching a lighthearted comedy The plot is simplistic but the dialogue is witty and the characters are likable even the well bread suspected serial killer While some may be disappointed when they realize this is not Match Point 2 Risk Addiction I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to lovebr br This was the most Id laughed at one of Woodys comedies in years dare I say a decade While Ive never been impressed with Scarlet Johanson in this she managed to tone down her sexy image and jumped right into a average but spirited young womanbr br This may not be the crown jewel of his career but it was wittier than Devil Wears Prada and more interesting than Superman a great comedy to go see with friends,"[i, thought, this, was, a, wonderful, way, to, spend, time, on, a, too, hot, summer, weekend, sitting, in, the, air, conditioned, theater, and, watching, a, lighthearted, comedy, the, plot, is, simplistic, but, the, dialogue, is, witty, and, the, characters, are, likable, even, the, well, bread, suspected, serial, killer, while, some, may, be, disappointed, when, they, realize, this, is, not, match, point, 2, risk, addiction, i, thought, it, was, proof, that, woody, allen, is, still, fully, in, control, of, the, style, many, of, us, have, grown, to, lovebr, br, this, was, the, most, id, laughed, at, one, of, woodys, comedies, in, ...]"
1,"Probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a noble cause, but it's not preachy or boring. It just never gets old, despite my having seen it some 15 or more times in the last 25 years. Paul Lukas' performance brings tears to my eyes, and Bette Davis, in one of her very few truly sympathetic roles, is a delight. The kids are, as grandma says, more like ""dressed-up midgets"" than children, but that only makes them more fun to watch. And the mother's slow awakening to what's happening in the world and under her own roof is believable and startling. If I had a dozen thumbs, they'd all be ""up"" for this movie.",positive,Probably my alltime favorite movie a story of selflessness sacrifice and dedication to a noble cause but its not preachy or boring It just never gets old despite my having seen it some 15 or more times in the last 25 years Paul Lukas performance brings tears to my eyes and Bette Davis in one of her very few truly sympathetic roles is a delight The kids are as grandma says more like dressedup midgets than children but that only makes them more fun to watch And the mothers slow awakening to whats happening in the world and under her own roof is believable and startling If I had a dozen thumbs theyd all be up for this movie,"[probably, my, alltime, favorite, movie, a, story, of, selflessness, sacrifice, and, dedication, to, a, noble, cause, but, its, not, preachy, or, boring, it, j

***

**OBSERVATION**

* Tokenization of `body_text_clean_punc` column is performed.
* By **Tokenizing** , each words are counted.
* Also, the values are converted to lower cases. 
* The new tokenized values in lower case  are stored in `body_text_tokenized`. 

***


### 5. Remove stopwords:

<a name="stop"></a>

In [23]:
import nltk
stopword = nltk.corpus.stopwords.words('english')

In [24]:
# stopwords removal process
def remove_stopwords(tokenized_list):
    text = [word for word in tokenized_list if word not in stopword]
    return text

df2['body_text_nostop'] = df2['body_text_tokenized'].apply(lambda x: remove_stopwords(x))

df2.head()

,review,sentiment,body_text_clean_punc,body_text_tokenized,body_text_nostop
0,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her ""sexy"" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than ""Devil Wears Prada"" and more interesting than ""Superman"" a great comedy to go see with friends.",positive,I thought this was a wonderful way to spend time on a too hot summer weekend sitting in the air conditioned theater and watching a lighthearted comedy The plot is simplistic but the dialogue is witty and the characters are likable even the well bread suspected serial killer While some may be disappointed when they realize this is not Match Point 2 Risk Addiction I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to lovebr br This was the most Id laughed at one of Woodys comedies in years dare I say a decade While Ive never been impressed with Scarlet Johanson in this she managed to tone down her sexy image and jumped right into a average but spirited young womanbr br This may not be the crown jewel of his career but it was wittier than Devil Wears Prada and more interesting than Superman a great comedy to go see with friends,"[i, thought, this, was, a, wonderful, way, to, spend, time, on, a, too, hot, summer, weekend, sitting, in, the, air, conditioned, theater, and, watching, a, lighthearted, comedy, the, plot, is, simplistic, but, the, dialogue, is, witty, and, the, characters, are, likable, even, the, well, bread, suspected, serial, killer, while, some, may, be, disappointed, when, they, realize, this, is, not, match, point, 2, risk, addiction, i, thought, it, was, proof, that, woody, allen, is, still, fully, in, control, of, the, style, many, of, us, have, grown, to, lovebr, br, this, was, the, most, id, laughed, at, one, of, woodys, comedies, in, ...]","[thought, wonderful, way, spend, time, hot, summer, weekend, sitting, air, conditioned, theater, watching, lighthearted, comedy, plot, simplistic, dialogue, witty, characters, likable, even, well, bread, suspected, serial, killer, may, disappointed, realize, match, point, 2, risk, addiction, thought, proof, woody, allen, still, fully, control, style, many, us, grown, lovebr, br, id, laughed, one, woodys, comedies, years, dare, say, decade, ive, never, impressed, scarlet, johanson, managed, tone, sexy, image, jumped, right, average, spirited, young, womanbr, br, may, crown, jewel, career, wittier, devil, wears, prada, interesting, superman, great, comedy, go, see, friends]"
1,"Probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a noble cause, but it's not preachy or boring. It just never gets old, despite my having seen it some 15 or more times in the last 25 years. Paul Lukas' performance brings tears to my eyes, and Bette Davis, in one of her very few truly sympathetic roles, is a delight. The kids are, as grandma says, more like ""dressed-up midgets"" than children, but that only makes them more fun to watch. And the mother's slow awakening to what's happening in the world and under her own roof is believable and startling. If I had a dozen thumbs, they'd all be ""up"" for this movie.",positive,Probably my alltime favorite movie a story of selflessness sacrifice and dedication to a 

<hr style="height:1px">

**OBSERVATION**

* Stopwords are removed from `body_text_tokenized` column . 
* **Stopwords** are normal words like articles. We can see that  pronouns like I, was, on, this are removed from tokenized strings.
* The new values without stopwords are stored in `body_text_nostop`. 

<hr style="height:1px">

### 6. Stemming:

<a name="stem"></a>

In [25]:
#stemming process
import nltk

ps = nltk.PorterStemmer()

def stemming(tokenized_text):
    text = [ps.stem(word) for word in tokenized_text]
    return text

df2['body_text_stemmed'] = df2['body_text_nostop'].apply(lambda x: stemming(x))

df2.head()

,review,sentiment,body_text_clean_punc,body_text_tokenized,body_text_nostop,body_text_stemmed
0,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her ""sexy"" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than ""Devil Wears Prada"" and more interesting than ""Superman"" a great comedy to go see with friends.",positive,I thought this was a wonderful way to spend time on a too hot summer weekend sitting in the air conditioned theater and watching a lighthearted comedy The plot is simplistic but the dialogue is witty and the characters are likable even the well bread suspected serial killer While some may be disappointed when they realize this is not Match Point 2 Risk Addiction I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to lovebr br This was the most Id laughed at one of Woodys comedies in years dare I say a decade While Ive never been impressed with Scarlet Johanson in this she managed to tone down her sexy image and jumped right into a average but spirited young womanbr br This may not be the crown jewel of his career but it was wittier than Devil Wears Prada and more interesting than Superman a great comedy to go see with friends,"[i, thought, this, was, a, wonderful, way, to, spend, time, on, a, too, hot, summer, weekend, sitting, in, the, air, conditioned, theater, and, watching, a, lighthearted, comedy, the, plot, is, simplistic, but, the, dialogue, is, witty, and, the, characters, are, likable, even, the, well, bread, suspected, serial, killer, while, some, may, be, disappointed, when, they, realize, this, is, not, match, point, 2, risk, addiction, i, thought, it, was, proof, that, woody, allen, is, still, fully, in, control, of, the, style, many, of, us, have, grown, to, lovebr, br, this, was, the, most, id, laughed, at, one, of, woodys, comedies, in, ...]","[thought, wonderful, way, spend, time, hot, summer, weekend, sitting, air, conditioned, theater, watching, lighthearted, comedy, plot, simplistic, dialogue, witty, characters, likable, even, well, bread, suspected, serial, killer, may, disappointed, realize, match, point, 2, risk, addiction, thought, proof, woody, allen, still, fully, control, style, many, us, grown, lovebr, br, id, laughed, one, woodys, comedies, years, dare, say, decade, ive, never, impressed, scarlet, johanson, managed, tone, sexy, image, jumped, right, average, spirited, young, womanbr, br, may, crown, jewel, career, wittier, devil, wears, prada, interesting, superman, great, comedy, go, see, friends]","[thought, wonder, way, spend, time, hot, summer, weekend, sit, air, condit, theater, watch, lightheart, comedi, plot, simplist, dialogu, witti, charact, likabl, even, well, bread, suspect, serial, killer, may, disappoint, realiz, match, point, 2, risk, addict, thought, proof, woodi, allen, still, fulli, control, style, mani, us, grown, lovebr, br, id, laugh, one, woodi, comedi, year, dare, say, decad, ive, never, impress, scarlet, johanson, manag, tone, sexi, imag, jump, right, averag, spirit, young, womanbr, br, may, crown, jewel, career, wittier, devil, wear, prada, interest, superman, great, comedi, go, see, friend]"
1,"Probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a noble cause, but it's

<hr style="height:1px">

**OBSERVATION**

* Stemming is perfeormed on removed from `body_text_nostop` column . 
* **Stemming** are words that remove the latter half of a word such as -ing , ly. 
* The new values are stored in `body_text_stemmed`.
* Stemming abruptly removes last half of the words and lose their originality. 

<hr style="height:1px">


### 7. Lemmatizing:

<a name="lemma"></a>

In [26]:
# import lemmatizer
import nltk
lm = nltk.WordNetLemmatizer()

In [27]:
#lemmatzing
def lemmatizing(tokenized_text):
    text = [lm.lemmatize(word) for word in tokenized_text]
    return text

df2['body_text_lemmatized'] = df2['body_text_nostop'].apply(lambda x: lemmatizing(x))

df2.head(10)

,review,sentiment,body_text_clean_punc,body_text_tokenized,body_text_nostop,body_text_stemmed,body_text_lemmatized
0,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her ""sexy"" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than ""Devil Wears Prada"" and more interesting than ""Superman"" a great comedy to go see with friends.",positive,I thought this was a wonderful way to spend time on a too hot summer weekend sitting in the air conditioned theater and watching a lighthearted comedy The plot is simplistic but the dialogue is witty and the characters are likable even the well bread suspected serial killer While some may be disappointed when they realize this is not Match Point 2 Risk Addiction I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to lovebr br This was the most Id laughed at one of Woodys comedies in years dare I say a decade While Ive never been impressed with Scarlet Johanson in this she managed to tone down her sexy image and jumped right into a average but spirited young womanbr br This may not be the crown jewel of his career but it was wittier than Devil Wears Prada and more interesting than Superman a great comedy to go see with friends,"[i, thought, this, was, a, wonderful, way, to, spend, time, on, a, too, hot, summer, weekend, sitting, in, the, air, conditioned, theater, and, watching, a, lighthearted, comedy, the, plot, is, simplistic, but, the, dialogue, is, witty, and, the, characters, are, likable, even, the, well, bread, suspected, serial, killer, while, some, may, be, disappointed, when, they, realize, this, is, not, match, point, 2, risk, addiction, i, thought, it, was, proof, that, woody, allen, is, still, fully, in, control, of, the, style, many, of, us, have, grown, to, lovebr, br, this, was, the, most, id, laughed, at, one, of, woodys, comedies, in, ...]","[thought, wonderful, way, spend, time, hot, summer, weekend, sitting, air, conditioned, theater, watching, lighthearted, comedy, plot, simplistic, dialogue, witty, characters, likable, even, well, bread, suspected, serial, killer, may, disappointed, realize, match, point, 2, risk, addiction, thought, proof, woody, allen, still, fully, control, style, many, us, grown, lovebr, br, id, laughed, one, woodys, comedies, years, dare, say, decade, ive, never, impressed, scarlet, johanson, managed, tone, sexy, image, jumped, right, average, spirited, young, womanbr, br, may, crown, jewel, career, wittier, devil, wears, prada, interesting, superman, great, comedy, go, see, friends]","[thought, wonder, way, spend, time, hot, summer, weekend, sit, air, condit, theater, watch, lightheart, comedi, plot, simplist, dialogu, witti, charact, likabl, even, well, bread, suspect, serial, killer, may, disappoint, realiz, match, point, 2, risk, addict, thought, proof, woodi, allen, still, fulli, control, style, mani, us, grown, lovebr, br, id, laugh, one, woodi, comedi, year, dare, say, decad, ive, never, impress, scarlet, johanson, manag, tone, sexi, imag, jump, right, averag, spirit, young, womanbr, br, may, crown, jewel, career, wittier, devil, wear, prada, interest, superman, great, comedi, go, see, friend]","[thought, wonderful, way, spend, time, hot, summer, weekend, sitting, air, conditioned, theate

In [28]:
#Label coding 0 and 1
df2['sentiment'].replace({'negative':0,'positive':1},inplace=True)

In [29]:
df2.head()

,review,sentiment,body_text_clean_punc,body_text_tokenized,body_text_nostop,body_text_stemmed,body_text_lemmatized
0,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her ""sexy"" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than ""Devil Wears Prada"" and more interesting than ""Superman"" a great comedy to go see with friends.",1,I thought this was a wonderful way to spend time on a too hot summer weekend sitting in the air conditioned theater and watching a lighthearted comedy The plot is simplistic but the dialogue is witty and the characters are likable even the well bread suspected serial killer While some may be disappointed when they realize this is not Match Point 2 Risk Addiction I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to lovebr br This was the most Id laughed at one of Woodys comedies in years dare I say a decade While Ive never been impressed with Scarlet Johanson in this she managed to tone down her sexy image and jumped right into a average but spirited young womanbr br This may not be the crown jewel of his career but it was wittier than Devil Wears Prada and more interesting than Superman a great comedy to go see with friends,"[i, thought, this, was, a, wonderful, way, to, spend, time, on, a, too, hot, summer, weekend, sitting, in, the, air, conditioned, theater, and, watching, a, lighthearted, comedy, the, plot, is, simplistic, but, the, dialogue, is, witty, and, the, characters, are, likable, even, the, well, bread, suspected, serial, killer, while, some, may, be, disappointed, when, they, realize, this, is, not, match, point, 2, risk, addiction, i, thought, it, was, proof, that, woody, allen, is, still, fully, in, control, of, the, style, many, of, us, have, grown, to, lovebr, br, this, was, the, most, id, laughed, at, one, of, woodys, comedies, in, ...]","[thought, wonderful, way, spend, time, hot, summer, weekend, sitting, air, conditioned, theater, watching, lighthearted, comedy, plot, simplistic, dialogue, witty, characters, likable, even, well, bread, suspected, serial, killer, may, disappointed, realize, match, point, 2, risk, addiction, thought, proof, woody, allen, still, fully, control, style, many, us, grown, lovebr, br, id, laughed, one, woodys, comedies, years, dare, say, decade, ive, never, impressed, scarlet, johanson, managed, tone, sexy, image, jumped, right, average, spirited, young, womanbr, br, may, crown, jewel, career, wittier, devil, wears, prada, interesting, superman, great, comedy, go, see, friends]","[thought, wonder, way, spend, time, hot, summer, weekend, sit, air, condit, theater, watch, lightheart, comedi, plot, simplist, dialogu, witti, charact, likabl, even, well, bread, suspect, serial, killer, may, disappoint, realiz, match, point, 2, risk, addict, thought, proof, woodi, allen, still, fulli, control, style, mani, us, grown, lovebr, br, id, laugh, one, woodi, comedi, year, dare, say, decad, ive, never, impress, scarlet, johanson, manag, tone, sexi, imag, jump, right, averag, spirit, young, womanbr, br, may, crown, jewel, career, wittier, devil, wear, prada, interest, superman, great, comedi, go, see, friend]","[thought, wonderful, way, spend, time, hot, summer, weekend, sitting, air, conditioned, theater, watc

***
**OBSERVATION**

* Lemmatizing is performed on removed from `body_text_nostop` column . 
* **Lemmatizing** are words that remove the latter only if necessary,else retain's the original word as is. 
* The new values are stored in `body_text_lemmatized`.
* Finally, the target variable `Sentiment` is label encoded . 0 =  Negative && 1 = Positive

***

<hr style="border:01px  solid gray"> </hr>

## <center>DATA MODELLING<a name="dm"></a>

<hr style="border:01px  solid gray"> </hr>

In DataModelling, the following steps are performed
1. [Train_test_split](#tts)
2. [Gridsearch CV](#gscv)
3. [Model Evaluation](#me)

###  1. Train_Test_Split: <a name="tts"></a>

In [30]:
#splitting dataset to train & test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df2[['review']], df2['sentiment'], test_size=0.2)

In [31]:
stopwords = nltk.corpus.stopwords.words('english')

In [32]:
# cleaning the input
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

In [33]:
#applying TFID 
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer= clean_text)
tfidf_vect_fit = tfidf_vect.fit(X_train['review'])


In [34]:
#fit_transform X_train & X_test
tfidf_train = tfidf_vect_fit.transform(X_train['review'])
tfidf_test = tfidf_vect_fit.transform(X_test['review'])

In [35]:
X_train_vect = pd.DataFrame(tfidf_train.toarray())
X_test_vect =pd.DataFrame(tfidf_test.toarray())

X_train_vect.head()

,0,1,2,3,4,5,6,7,8,9,...,37005,37006,37007,37008,37009,37010,37011,37012,37013,37014
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<hr style="border:.03px  solid black"> </hr>

**OBSERVATION**

* With **Train_Test_Split**, the train set is 80% and test set is 20%. 
* Term Frequency - Inverse Document frequency , is applied on the raw `review` column from the originial df. 
* The column `Review` is cleaned again by removing Punctuations, stopwords and tokenizing them followed by Stemming. 

<hr style="height:1px">

###  2.  Gridsearch CV: <a name="gscv"></a>

In [36]:
#import lib for gridsearchCv 
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [37]:
#Random Forest
rf = RandomForestClassifier()
param = {'n_estimators': [10, 50, 100],
        'max_depth': [20, 40, 60, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_train_vect, y_train)
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
8,53.348804,0.657138,0.616958,0.039512,60,100,"{'max_depth': 60, 'n_estimators': 100}",0.818750,0.838542,0.843750,0.827083,0.842708,0.834167,0.009714,1
11,55.023217,6.783977,0.660163,0.154113,None,100,"{'max_depth': None, 'n_estimators': 100}",0.820833,0.845833,0.844792,0.837500,0.819792,0.833750,0.011346,2
5,47.838822,1.097365,0.624336,0.050252,40,100,"{'max_depth': 40, 'n_estimators': 100}",0.828125,0.836458,0.841667,0.819792,0.833333,0.831875,0.007471,3
2,34.610166,2.010138,0.616776,0.064337,20,100,"{'max_depth': 20, 'n_estimators': 100}",0.820833,0.833333,0.835417,0.819792,0.826042,0.827083,0.006353,4
10,30.977119,0.646137,0.681274,0.104750,None,50,"{'max_depth': None, 'n_estimators': 50}",0.796875,0.826042,0.827083,0.817708,0.813542,0.816250,0.010937,5


In [38]:
#Gradient Boosting
gb = GradientBoostingClassifier()
param = {
    'n_estimators': [10, 20], 
    'max_depth': [7, 11, 15],
    'learning_rate': [0.1]
}

clf = GridSearchCV(gb, param, cv=5, n_jobs=-1)
cv_fit = clf.fit(X_train_vect, y_train)
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,795.230235,8.693441,0.541208,0.066363,0.1,11,20,"{'learning_rate': 0.1, 'max_depth': 11, 'n_estimators': 20}",0.740625,0.790625,0.771875,0.767708,0.754167,0.765000,0.016869,1
1,658.677442,14.647976,0.540389,0.021238,0.1,7,20,"{'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 20}",0.746875,0.791667,0.756250,0.766667,0.762500,0.764792,0.015000,2
5,815.392334,142.779744,0.509837,0.114180,0.1,15,20,"{'learning_rate': 0.1, 'max_depth': 15, 'n_estimators': 20}",0.740625,0.793750,0.754167,0.756250,0.764583,0.761875,0.017697,3
2,370.378104,6.720737,0.556031,0.061133,0.1,11,10,"{'learning_rate': 0.1, 'max_depth': 11, 'n_estimators': 10}",0.732292,0.769792,0.734375,0.752083,0.736458,0.745000,0.014228,4
4,413.661825,10.430789,0.552181,0.016792,0.1,15,10,"{'learning_rate': 0.1, 'max_depth': 15, 'n_estimators': 10}",0.722917,0.777083,0.731250,0.742708,0.735417,0.741875,0.018731,5


<hr style="height:1px">

**OBSERVATION**

* From GridSearchCV for Random Forest, the best_param chosen is `{'max_depth': 40, 'n_estimators': 100}` , which yeilds a mean test score of 83.1% as it provides a less mean time.

*  From GridSearchCV for Gradient Boosting, the best_param chosen is `{'learning_rate': 0.1, 'max_depth': 11, 'n_estimators': 10}` , which yeilds a mean test score of 74.5% and has the least fitting time. 

<hr style="height:1px">


### 3.Model Evaluation :<a name="me"></a>

In [52]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support as score
import time

In [53]:
rf = RandomForestClassifier(n_estimators=100, max_depth=40, n_jobs=-1)

start = time.time()
rf_model = rf.fit(X_train_vect, y_train)
end = time.time()
fit_time = (end - start)

start = time.time()
y_pred = rf_model.predict(X_test_vect)
end = time.time()
pred_time = (end - start)

precision, recall, fscore, train_support = score(y_test, y_pred, pos_label= 1 , average='binary')
print('Fit time: {} / Predict time: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(fit_time, 3), round(pred_time, 3), round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))

Fit time: 14.164 / Predict time: 0.422 ---- Precision: 0.856 / Recall: 0.832 / Accuracy: 0.846


In [54]:
gb = GradientBoostingClassifier(n_estimators=10, max_depth= 11)

start = time.time()
gb_model = gb.fit(X_train_vect, y_train)
end = time.time()
fit_time = (end - start)

start = time.time()
y_pred = gb_model.predict(X_test_vect)
end = time.time()
pred_time = (end - start)

precision, recall, fscore, train_support = score(y_test, y_pred, pos_label= 1 , average='binary')
print('Fit time: {} / Predict time: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(fit_time, 3), round(pred_time, 3), round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))

Fit time: 202.167 / Predict time: 0.375 ---- Precision: 0.743 / Recall: 0.8 / Accuracy: 0.762


***

**OBSERVATION**

* In Model Evaluation, 
* With respect to **RandomForest** ,
    * when passed with best params found, we see RandomForest Produced 84.6% of accuracy. 
    * Fit time and predict time is as low as 14.164 and 0.412. 
    * We have passed `pos_label = 1` , 1 = positive .Therefore the recall value is considered which is 83% . 
* With respect to **GradientBoosting** ,
    * When passed with best params, we see the accuracy has dropped to 76.2%. But has a recall value of 80%.
    * But the time taken for the model to fit is more. 
    * Also,Prediction time is less as 0.375.
    

***

<hr style="border:01px  solid gray"> </hr>

## <center>CONCLUSION<a name="conc"></a>

<hr style="border:01px  solid gray"> </hr>

* Since it is a text document classification, we will be concentrating on `Recall_Scores` as opposed to `Accuracy`. As this project aims at classifying the total relevant results correctly . 
* With Lemmatizing, a lot of time was taken to compute. Hence, to reduce the fitting time,stemming was used in this dataset where  45356 Lemma's were reduced to 30k words with stemming.
* Since the dataset is large, time taken to run gridsearchCV with more parameters was minimized. 
* Also, with regards to `pos_label = 1` ,**recall** is considered for this data modelling. 
* We can infer that both , Random Forest and Gradient Boosting provided a `recall` of 83% & 80% respectively.
* However, the `predict_time` was **less in GradientBoosting** when compared to RandomForest.
